In [70]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
%config Completer.use_jedi = False  # 자동완성 속도 증가

In [4]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(
    mean=[0, 3],
    cov=[[1, 0.5], [0.5, 1]],
    size=num_samples_per_class)
positive_samples = np.random.multivariate_normal(
    mean=[3, 0],
    cov=[[1, 0.5], [0.5, 1]],
    size=num_samples_per_class)

In [5]:
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype='float32'),
                   np.ones((num_samples_per_class, 1), dtype='float32')))

In [6]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value) # 랜덤한 값으로 초기화된 (input_size, output_size)크기의 행렬 W를 만듬
        
        b_shape = (output_size)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value) # 0으로 초기화된 (outputsize,) 크기의 벡터 b를 만듬
    
    def __call__(self, inputs): # 정방향 패스를 수행
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self): # 가중치를 추출
        return [self.W, self.b]

In [7]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [8]:
# Layer의 서브클래스로 구현한 Dense층
from tensorflow import keras

class SimpleDense(keras.layers.Layer): # 모든 Keras층은 Layer 클래스를 상속
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation
        
    def build(self, input_shape): # build() 메서드에서 가중치를 생성
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units),
                                initializer='random_normal')
        self.b = self.add_weight(shape=(self.units,),
                                initializer='zeros')
        
    def call(self, inputs): # call() 메서드에서 정방향 패스 계산을 정의
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [9]:
from tensorflow.keras import layers
layer = layers.Dense(32, activation='relu')

In [10]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential([
    layers.Dense(32, activation='relu'),
    layers.Dense(32)
])

In [11]:
model = NaiveSequential([
    NaiveDense(input_size=784, output_size=32, activation='relu'),
    NaiveDense(input_size=32, output_size=64, activation='relu'),
    NaiveDense(input_size=64, output_size=32, activation='relu'),
    NaiveDense(input_size=32, output_size=10, activation='softmax')
])

In [12]:
model = keras.Sequential([
    SimpleDense(32, activation='relu'),
    SimpleDense(64, activation='relu'),
    SimpleDense(32, activation='relu'),
    SimpleDense(10, activation='softmax')
])

In [13]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])

In [14]:
history = model.fit(
    inputs,
    targets,
    epochs=5,
    batch_size=128
)
history.history

Epoch 1/5
2000/2000 [==============================] - 0s 32us/sample - loss: 12.2885 - acc: 0.4065
Epoch 2/5
2000/2000 [==============================] - 0s 5us/sample - loss: 11.8282 - acc: 0.4040
Epoch 3/5
2000/2000 [==============================] - 0s 6us/sample - loss: 11.4435 - acc: 0.3990
Epoch 4/5
2000/2000 [==============================] - 0s 5us/sample - loss: 11.0719 - acc: 0.3955
Epoch 5/5
2000/2000 [==============================] - 0s 5us/sample - loss: 10.7062 - acc: 0.3930


{'loss': [12.288459770202637,
  11.8282066116333,
  11.443466102600098,
  11.071913749694824,
  10.706180892944335],
 'acc': [0.4065, 0.404, 0.399, 0.3955, 0.393]}

In [67]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
             loss=keras.losses.MeanSquaredError(),
             metrics=[keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

In [68]:
model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets)
)
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)
print(loss_and_metrics)

Train on 1400 samples, validate on 600 samples
Epoch 1/5
1400/1400 [==============================] - 0s 153us/sample - loss: 0.3640 - binary_accuracy: 0.9636 - val_loss: 0.0670 - val_binary_accuracy: 0.9600
Epoch 2/5
1400/1400 [==============================] - 0s 63us/sample - loss: 0.0717 - binary_accuracy: 0.9600 - val_loss: 0.0726 - val_binary_accuracy: 0.9383
Epoch 3/5
1400/1400 [==============================] - 0s 70us/sample - loss: 0.0722 - binary_accuracy: 0.9564 - val_loss: 0.0919 - val_binary_accuracy: 0.9950
Epoch 4/5
1400/1400 [==============================] - 0s 68us/sample - loss: 0.0710 - binary_accuracy: 0.9607 - val_loss: 0.1094 - val_binary_accuracy: 0.8683
Epoch 5/5
600/600 [==============================] - 0s 48us/sample - loss: 0.1609 - binary_accuracy: 0.8250
[0.160890957514445, 0.825]


In [69]:
predictions = model.predict(val_inputs, batch_size=128)
print(predictions[:10])

[[ 0.69090927]
 [ 0.64283186]
 [-0.3483867 ]
 [-0.4437498 ]
 [-0.32302064]
 [ 0.22651562]
 [ 0.21623285]
 [-0.30339772]
 [-0.14156371]
 [-0.22358489]]
